In [6]:
# import the required package 
import statistics
import yfinance as yt
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import yahoo_fin
import yfinance as yt
from yahoo_fin import stock_info as si
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from datetime import date, datetime
# create a list for individual risk,individual return, portfolio risk, portfolio return
indi_ret=[]
indi_risk=[]
port_ret=[]
portfo_risk=[]


In [7]:
# create a class for portfolio by passing no of company ,weights,ticker
class Portfolio():
    # here we are intilise the variable which are used under the this class portfolio 
    def __init__(self, n_companies, weights, ticker):
        self.n_companies = n_companies
        self.weights = weights
        self.ticker = ticker
        self.returns, self.exp_returns, self.risk = self.risk_return_individual()
       
    def risk_return_individual(self):
        # here we are create a function for individual risk and return 
        risk = [] 
        exp_returns = []
        returns = []
        
        for i in range(len(self.ticker)):
            data = yt.download(self.ticker[i])
            ret = []
            lst = data['Close']
            for j in range(len(lst)-1):
                ret.append(((lst[j+1]-lst[j])/lst[j])*100)
            returns.append(ret)# for return we want to do current close price - previous close price/previous close price
            exp_returns.append(statistics.mean(ret))#for expected return we want to do mean of return
            risk.append(statistics.stdev(ret))#for risk we want to do standard deviation of return
        print('\nRisk of the individual companies : ',risk)
        indi_ret.append([exp_returns])
        indi_risk.append([risk])
        return returns, exp_returns, risk
    def portfolio_return(self):
        #create a function for generate portfolio return
        port_returns = sum([a*b for a,b in zip(self.weights, self.exp_returns)])# for portfolio return we want to do sum(weight multiply with expected return)
        port_ret.append([port_returns])
        print('Portfolio expected return is : ',port_returns)
    def portfolio_risk(self):
        #create a function for generate portfolio risk
        term_1 = sum([w*w*r*r for w, r in zip(self.weights,self.risk)])# for portfolio risk term1 we want to do sum(weight square multiply with expected return square)
        # here we are creating matrix with zero 
        cov_matrix = np.zeros((self.n_companies,self.n_companies))
        for i in range(self.n_companies):
            for j in range(self.n_companies):
                min_len = min(len(self.returns[i]), len(self.returns[j]))
                cov_matrix[i][j] = np.cov(self.returns[i][-min_len:],self.returns[j][-min_len:], ddof = 1)[0][1]
        term_2 = 2*(sum([self.weights[i]*self.weights[j]*cov_matrix[i][j] for i in range(self.n_companies) for j in range(i+1, self.n_companies)]))
        port_risk = (term_1 + term_2)**0.5
        portfo_risk.append([port_risk])
        print('Portfolio risk is :', port_risk)  
ticker=[]



In [8]:
# for passing value to portfolio class (n-com,weights,ticker)
n_com=[]
weightss=[]
#here we are getting (n-com,weights,ticker) every thing from user from the user interest it produce individual risk,individual return, portfolio risk, portfolio return
get_por=int(input('enter the no of portfolio count: '))
for i in range(get_por):
    n_companies = int(input('Enter the number of companies in your portfolio : '))
    n_com.append(n_companies)
    tc=[]
    ticker.append([tc])
    for i in range(n_companies):
        g=input("enter a company name : ")
        tc.append(g)
    weights = []
    weightss.append(weights)
    for i in range(n_companies):
        weights.append(float(input(f'Enter your weights of the  companies {tc[i]} : ')))
    x = Portfolio(n_companies, weights,tc)
    x.portfolio_return()
    x.portfolio_risk()

enter the no of portfolio count: 1
Enter the number of companies in your portfolio : 2
enter a company name : ibm
enter a company name : goog
Enter your weights of the  companies ibm : 0.8
Enter your weights of the  companies goog : 0.2
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Risk of the individual companies :  [1.591392178861302, 1.9439851175631495]
Portfolio expected return is :  0.045346600645564686
Portfolio risk is : 1.4603275810043541


In [33]:
# here we are asking user to no of company 
n_com=[]
tc=[]
c=int(input('Enter the number of ticker: '))
for k in range(c):
    g=input("enter a company name : ")
    tc.append(g)
ff=[]
for kk in tc:
    ff.append(kk)

#we are getting ticker from user and we download it and store in stock.csv
st= yt.download(ff[0])
stock_data=st.to_csv('stock.csv')
   
st= yt.download(ff[1])
stock_data=st.to_csv('stock1.csv')

Enter the number of ticker: 2
enter a company name : ibm
enter a company name : goog
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [34]:
import pandas as pd
files = ['stock.csv', 'stock1.csv']
df = pd.DataFrame()
for file in files:
    data = pd.read_csv(file)
    df = pd.concat([df, data], axis=0)
df.to_csv('merged_files.csv', index=False)

In [38]:
# # Load the stock data for perform AE+LSTM+OMEGA
#import the required package 
import numpy as np
import math
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import date
from tensorflow.keras.layers import Input
a=(0.6,0.4)
# Load the stock data
stock_data = pd.read_csv('merged_files.csv') # Replace with your dataset
print(stock_data.head())

# we are taking close price of each stock
# Preprocess the data before perform program 
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(stock_data[['Close']].values)

# Split the data into training and testing sets
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

# Reshape the training data
train_data = np.reshape(train_data, (train_data.shape[0], 1))

# Define the autoencoder model with dimension 
input_dim = train_data.shape[1]
encoding_dim = 64
autoencoder = Sequential()
autoencoder.add(Dense(encoding_dim, activation='relu', input_shape=(input_dim,)))
autoencoder.add(Dense(input_dim, activation='linear'))

#  Define the Omega model
autoencoder_input =Input(shape=(input_dim,))
encoded_data = autoencoder(autoencoder_input)

# Define the model
timesteps = 10
lstm_units = 64

model = Sequential()
model.add(LSTM(lstm_units, activation='relu', input_shape=(timesteps, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(train_data, train_data, epochs=10)
lstm_input = RepeatVector(timesteps)(encoded_data)
predicted_data = model(lstm_input)

omega_model = Model(autoencoder_input, predicted_data)

# Compile and train the Omega model
omega_model.compile(optimizer='adam', loss='mean_squared_error')
omega_model.fit(train_data[:-timesteps], train_data[timesteps:], epochs=10, batch_size=32)

# Generate predictions on the test data
encoded_test_data = autoencoder.predict(test_data  [:-timesteps])
predicted_test_data = model.predict(np.repeat(encoded_test_data, timesteps, axis=0))
predicted_test_data = np.reshape(predicted_test_data, (-1, 1))

# Scale the predicted data back to the original range
predicted_test_data = scaler.inverse_transform(predicted_test_data)
# Evaluate the model
test_actual_data= stock_data['Close'].values[train_size + timesteps:]
d=test_actual_data-predicted_test_data
D=d.flatten()
mse = np.mean(D**2)
rmse=math.sqrt(mse)
print("Mean Squared Error:",mse)
print("Root Mean Squared Error:",rmse)


         Date      Open      High       Low     Close  Adj Close  Volume
0  1962-01-02  7.374124  7.374124  7.291268  7.291268   1.591516  407940
1  1962-01-03  7.291268  7.355003  7.291268  7.355003   1.605428  305955
2  1962-01-04  7.355003  7.355003  7.278521  7.281708   1.589430  274575
3  1962-01-05  7.272148  7.272148  7.125558  7.138305   1.558128  384405
4  1962-01-08  7.131931  7.131931  6.947100  7.004461   1.528914  572685
Epoch 1/10
505/505 [==============================] - 4s 3ms/step - loss: 0.0250
Epoch 2/10
505/505 [==============================] - 2s 3ms/step - loss: 1.8688e-04
Epoch 3/10
505/505 [==============================] - 2s 3ms/step - loss: 1.4092e-04
Epoch 4/10
505/505 [==============================] - 2s 3ms/step - loss: 9.1540e-05
Epoch 5/10
505/505 [==============================] - 2s 3ms/step - loss: 4.7264e-05
Epoch 6/10
505/505 [==============================] - 2s 3ms/step - loss: 1.7118e-05
Epoch 7/10
505/505 [==============================] - 2s

In [29]:
x=Portfolio(c,a,tc)
x.portfolio_return()
x.portfolio_risk()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Risk of the individual companies :  [1.591392178861302, 1.9439851175631495]
Portfolio expected return is :  0.059423779917789465
Portfolio risk is : 1.4343030848914706


In [35]:
pd.read_csv('stock.csv')

,Date,Open,High,Low,Close,Adj Close,Volume
0,1962-01-02,7.374124,7.374124,7.291268,7.291268,1.591516,407940
1,1962-01-03,7.291268,7.355003,7.291268,7.355003,1.605428,305955
2,1962-01-04,7.355003,7.355003,7.278521,7.281708,1.589430,274575
3,1962-01-05,7.272148,7.272148,7.125558,7.138305,1.558128,384405
4,1962-01-08,7.131931,7.131931,6.947100,7.004461,1.528914,572685
...,...,...,...,...,...,...,...
15452,2023-05-22,127.500000,128.190002,127.150002,127.500000,127.500000,2806800
15453,2023-05-23,127.239998,129.089996,127.129997,128.179993,128.179993,4592300
15454,2023-05-24,127.820000,127.900002,125.470001,125.680000,125.680000,3915500
15455,2023-05-25,125.610001,127.230003,125.010002,126.760002,126.760002,4102900


In [36]:
pd.read_csv('stock1.csv')

,Date,Open,High,Low,Close,Adj Close,Volume
0,2004-08-19,2.490664,2.591785,2.390042,2.499133,2.499133,897427216
1,2004-08-20,2.515820,2.716817,2.503118,2.697639,2.697639,458857488
2,2004-08-23,2.758411,2.826406,2.716070,2.724787,2.724787,366857939
3,2004-08-24,2.770615,2.779581,2.579581,2.611960,2.611960,306396159
4,2004-08-25,2.614201,2.689918,2.587302,2.640104,2.640104,184645512
...,...,...,...,...,...,...,...
4721,2023-05-22,123.510002,127.050003,123.449997,125.870003,125.870003,29760200
4722,2023-05-23,124.930000,125.419998,123.050003,123.290001,123.290001,24477900
4723,2023-05-24,121.879997,122.750000,120.750000,121.639999,121.639999,23087900
4724,2023-05-25,125.209999,125.980003,122.900002,124.349998,124.349998,33812700


In [37]:
pd.read_csv('merged_files.csv')

,Date,Open,High,Low,Close,Adj Close,Volume
0,1962-01-02,7.374124,7.374124,7.291268,7.291268,1.591516,407940
1,1962-01-03,7.291268,7.355003,7.291268,7.355003,1.605428,305955
2,1962-01-04,7.355003,7.355003,7.278521,7.281708,1.589430,274575
3,1962-01-05,7.272148,7.272148,7.125558,7.138305,1.558128,384405
4,1962-01-08,7.131931,7.131931,6.947100,7.004461,1.528914,572685
...,...,...,...,...,...,...,...
20178,2023-05-22,123.510002,127.050003,123.449997,125.870003,125.870003,29760200
20179,2023-05-23,124.930000,125.419998,123.050003,123.290001,123.290001,24477900
20180,2023-05-24,121.879997,122.750000,120.750000,121.639999,121.639999,23087900
20181,2023-05-25,125.209999,125.980003,122.900002,124.349998,124.349998,33812700
